# ROS Basics - Part 2

so from the previous session, I recall I could not get much use from the turtulesim tutorial.
i wanted to take some look into the codes, see if it is in python and I catch anything from it.

so I load the ros container, I need to install the turtlesim package again.

searching in the packge files, looks like I can't find anything usefull :/


a few more ros commands in the turtlesim tut:
```
ros2 node info /my_turtle

ros2 run turtlesim turtlesim_node --ros-args --remap __node:=my_turtle


```
```ros2 topic list -t ```will return the same list of topics, this time with the topic type appended in brackets after each

```
/parameter_events [rcl_interfaces/msg/ParameterEvent]
/rosout [rcl_interfaces/msg/Log]
/turtle1/cmd_vel [geometry_msgs/msg/Twist]
/turtle1/color_sensor [turtlesim/msg/Color]
/turtle1/pose [turtlesim/msg/Pose]
```

To see the data being published on a topic, use: ```ros2 topic echo <topic_name> ```

Topics don’t have to only be point-to-point communication; it can be one-to-many, many-to-one, or many-to-many.
```
ros2 topic info /turtle1/cmd_vel
```

```
Type: geometry_msgs/msg/Twist
Publisher count: 1
Subscriber count: 2
```

Nodes send data over topics using messages. Publishers and subscribers must send and receive the same type of message to communicate.

The topic types we saw earlier after running ros2 topic list -t let us know what type of messages each topic can send. Recall that the cmd_vel topic has the type:
```
geometry_msgs/msg/Twist
```
Now we can run ```ros2 interface show <type>.msg ```on this type to learn its the details, specifically, what structure of data the message expects.


```
ros2 interface show geometry_msgs/msg/Twist

```

## set up a workspace

```
mkdir -p ~/dev_ws/src
cd ~/dev_ws/src
```

```
git clone https://github.com/ros/ros_tutorials.git -b <distro>-devel

```


installing the dependecies:

```
root@faff69968b4b:~/dev_ws# rosdep install -i --from-path src --rosdistro foxy -y
```

oaky the next step is to build the workspace with colcon

```
root@faff69968b4b:~/dev_ws# colcon build
```

>Note Other useful arguments for colcon build:

--packages-up-to builds the package you want, plus all its dependencies, but not the whole workspace (saves time)

--symlink-install saves you from having to rebuild every time you tweak python scripts

--event-handlers console_direct+ shows console output while building (can otherwise be found in the log directory)


The install directory is where your workspace’s setup files are, which you can use to source your overlay.

**Before sourcing the overlay, it is very important that you open a new terminal, separate from the one where you built the workspace. Sourcing an overlay in the same terminal where you built, or likewise building where an overlay is sourced, may create complex issues.**

okay so I can think of two ways for this issue, first put the build commands in the dockerfile, and second somehow reseting the terminal

okay I don't know if the issue I was seeing is related to the above comment or not but it is not working.
let me describe the error I gert:
[ugh I closed the container so I need to create it, and setup stuff, better yet to create a docker file]


**BTW** sh, ., bash, zsh, what is all the differece?!

okay so I run the dockerfile  and everything works fine until the last line:

RUN mkdir -p ~/dev_ws/src 

RUN cd ~/dev_ws/src &&  git clone https://github.com/ros/ros_tutorials.git -b foxy-devel

RUN apt-get update

RUN cd ~/dev_ws  && rosdep install -i --from-path src --rosdistro foxy -y

RUN cd ~/dev_ws  &&  colcon build

which gives this error:

```
[6/6] RUN cd ~/dev_ws  &&  colcon build:                                                                                                               
#9 0.823 Starting >>> turtlesim                                                                                                                           
#9 1.418 --- stderr: turtlesim                                                                                                                            
#9 1.418 CMake Error at CMakeLists.txt:12 (find_package):                                                                                                 
#9 1.418   By not providing "Findament_cmake.cmake" in CMAKE_MODULE_PATH this project                                                                     
#9 1.418   has asked CMake to find a package configuration file provided by
#9 1.418   "ament_cmake", but CMake did not find one.
#9 1.418 
#9 1.418   Could not find a package configuration file provided by "ament_cmake" with
#9 1.418   any of the following names:
#9 1.418 
#9 1.418     ament_cmakeConfig.cmake
#9 1.418     ament_cmake-config.cmake
#9 1.418 
#9 1.418   Add the installation prefix of "ament_cmake" to CMAKE_PREFIX_PATH or set
#9 1.418   "ament_cmake_DIR" to a directory containing one of the above files.  If
#9 1.418   "ament_cmake" provides a separate development package or SDK, be sure it
#9 1.418   has been installed.
#9 1.418 
#9 1.418 
#9 1.418 ---
#9 1.418 Failed   <<< turtlesim [0.59s, exited with code 1]
#9 1.453 
#9 1.453 Summary: 0 packages finished [0.82s]
#9 1.453   1 package failed: turtlesim
#9 1.453   1 package had stderr output: turtlesim

```


I tried running the same command inside the docker container and it works fine.
So searching for what could be solution?

okay so my search at [this](https://stackoverflow.com/questions/60545462/build-a-docker-image-ros2-when-try-tu-run-install-setup-bash-in-dockerfile-n) resulted in

that I should add these lines:

```
ENV ROS2_WS ~/dev_ws/

WORKDIR ${ROS2_WS}
```

which is odd I thought cd is changing the working directory

update: SO I tried building the image again with the same dockerfile. and this time it fails?!

on the other hand, I ran the 'colcon build' in the container, and then ran these two lines:

```
source /opt/ros/foxy/setup.bash
. install/local_setup.bash
```

which worked as expected this time.

So why the first did not work this and the second one worked?!

Anyway, I guess the 'source' command is not the way for the dockerfile.
I was to investigate the build proces of a package



okay so I went through the docker file for the demo package build in [here](https://hub.docker.com/_/ros) which is in ws_example_2

It is a multi stage build. So what I want is to adapt that docker file for my turtlesim docker.

okay so the new docker file is working:

this is how I could modify it:

```

ARG FROM_IMAGE=ros:foxy
ARG OVERLAY_WS=/opt/ros/overlay_ws

# multi-stage for caching
FROM $FROM_IMAGE AS cacher

# clone overlay source
ARG OVERLAY_WS
WORKDIR $OVERLAY_WS/src
RUN git clone https://github.com/ros/ros_tutorials.git -b foxy-devel


# multi-stage for building
FROM $FROM_IMAGE AS builder

# install overlay dependencies
ARG OVERLAY_WS
WORKDIR $OVERLAY_WS

COPY --from=cacher $OVERLAY_WS/src ./src

RUN . /opt/ros/$ROS_DISTRO/setup.sh && \
    apt-get update && rosdep install -i  -y \
      --from-path \
        src --rosdistro foxy 
  


RUN . /opt/ros/$ROS_DISTRO/setup.sh && \
    colcon build 
   


# source entrypoint setup
ENV OVERLAY_WS $OVERLAY_WS
RUN sed --in-place --expression \
      '$isource "$OVERLAY_WS/install/local_setup.bash"' \
      /ros_entrypoint.sh
```

But there are still unclear points from the original example that are left.
The last RUN command is unclear to me.

well I move on for this at the time being.